## RNA velocity computation
This script notebook uses velocyto and scvelo to compute the RNA velocity of a given scRNASeq dataset and display some of the results.  
velocyto documentation: http://velocyto.org/velocyto.py/index.html  
scvelo documentation: https://scvelo.readthedocs.io/  
scvelo paper: https://www.nature.com/articles/s41587-020-0591-3  

The working directory is Ahmad_workdir

In [ ]:
# Python libraries
!pip install python-igraph louvain
!pip install pybind11 hnswlib pysam
!pip install scvelo
!pip install velocyto
!pip install --user annoy
!pip install git+https://github.com/aiksi/vaevictis

In [ ]:
!conda install numpy scipy cython numba matplotlib scikit-learn h5py click --yes

In [ ]:
!conda install -c r -c bioconda pysam --yes

In [ ]:
!conda install -c bioconda samtools=1.15 --yes

In [ ]:
# This command did not work due to the data not having the exact structure velocyto excpects from a 10x experiment.
!velocyto run10x cellranger/Thymus2/outs/multi/count refdata-gex-GRCh38-2020-A/genes/genes.gtf

In [ ]:
# Syntax: velocyto run -o [output dir] [.bam file with spliced/unspliced data] [reference data] -b [cell barcode file]
!velocyto run -o scvelo_out cellranger/Thymus2/outs/per_sample_outs/Thymus2/count/sample_alignments.bam refdata-gex-GRCh38-2020-A/genes/genes.gtf
# Outputs a .loom file which will be used for downstream analysis

In [ ]:
# Imports
import scvelo as scv
import pandas as pd
from vaevictis import dimred
import os

scv.set_figure_params()
os.chdir('/data')

### Dataset #1: human thymus

In [ ]:
# read output loom file
# the loom filename is different for every velocyto output, check in the output folder
adata = scv.read('scvelo_out/sample_alignments_SHGNY.loom', cache=True)
adata.var_names_make_unique()
# The adata object is part of the anndata ecosystem. More details on how to handle such an object at https://adamgayoso.com/posts/ten_min_to_adata/

In [ ]:
# It is possible to filter the adata object using saved gene and cell barcodes
# However it is more efficient to pass a cell barcode file (.tsv.gz) directly as argument when calling velocyto
gene_inds = pd.read_csv('thy_filt_genes.csv', delimiter=',').values.tolist()
gene_inds = [g[0] for g in gene_inds]
cell_inds = pd.read_csv('thy_filt_cells.csv', delimiter=',').values.tolist()
cell_inds = [c[0] for c in cell_inds]
cell_inter = list(set(cell_inds).intersection(adata.obs_names))
gene_inter = list(set(gene_inds).intersection(adata.var_names))
adata = adata[cell_inter, gene_inter]

In [ ]:
# It is possible to compute vaevictis embeddings here, or to import embeddings computed beforehand (see later)

vae = dimred(adata.X.toarray(), 
             ww=[5., 0., 10., 5., 2., 1.],
             epochs=40,
             patience=4)
vae=vae[0]

In [ ]:
# Export embeddings to a file if needed
# .mtx is a marix sharing file format that can be read in R via Matrix::readMM()
scipy.io.mmwrite('scvelo_out/sample_alignments_SHGNY_red.mtx', vae[0])

In [ ]:
# Add embeddings to the adata object
adata.uns['vae'] = {'params': {'ww': [5., 0., 5., 5., 2., 1.]}} # not necessary
adata.obsm['X_vae'] = vae

In [ ]:
# preprocessing
scv.pp.filter_and_normalize(adata, min_shared_counts=20, n_top_genes=3000)
scv.pp.moments(adata, n_pcs=30, n_neighbors=30)
scv.tl.umap(adata) # umap not necessary if vaevictis has been used, but can be useful for a different visualization
# compute velocity
scv.tl.velocity(adata, mode='stochastic') # 3 possible modes: 'steady_state', 'dynamical', 'stochastic'
# build general graph
scv.pl.velocity_graph(adata, basis='vae')
# phase portraits of specific genes
# var_names is a list of any genes you want to focus on
# scv.pl.velocity(adata, var_names=['CD4', 'TOP2A'], basis='vae')
# compute latent time
scv.tl.recover_dynamics(adata)
scv.tl.latent_time(adata)

In [ ]:
# visualization (see docs for plot parameters)
# velocity projection
scv.pl.velocity_embedding_stream(adata, basis='vae')
# view latent time 
scv.pl.scatter(adata, color='latent_time', color_map='gnuplot', basis='vae')
# view pseudotime (similarity-based diffusion pseudotime)
scv.pl.scatter(adata, color='velocity_pseudotime', color_map='gnuplot', basis='vae')

### Dataset #2: mice organs
This dataset is comprised of 4 samples (DNWT, DNMUT, TOTWT, TOTMUT) that can be analyzed separately or merged together.  
The analyzed data corresponds to the most recent pre-processing iteration in mice_analysis_final.R

In [ ]:
# here the -b argument was used to specify a barcode file to filter cells 
!velocyto run -o scvelo_out/DNWT cellranger/Mice/08-23/DNWT/outs/per_sample_outs/DNWT/count/sample_alignments.bam mice_ref/genes.gtf -b velocyto_barcodes/DNWT_filt.tsv.gz

In [ ]:
!velocyto run -o scvelo_out/DNMUT cellranger/Mice/08-23/DNMUT/outs/per_sample_outs/DNMUT/count/sample_alignments.bam mice_ref/genes.gtf -b velocyto_barcodes/DNMUT_filt.tsv.gz

In [ ]:
!velocyto run -o scvelo_out/TOTWT cellranger/Mice/08-23/TOTWT/outs/per_sample_outs/TOTWT/count/sample_alignments.bam mice_ref/genes.gtf -b velocyto_barcodes/TOTWT_filt.tsv.gz

In [ ]:
!velocyto run -o scvelo_out/TOTMUT cellranger/Mice/08-23/TOTMUT/outs/per_sample_outs/TOTMUT/count/sample_alignments.bam mice_ref/genes.gtf -b velocyto_barcodes/TOTMUT_filt.tsv.gz

In [ ]:
# Import vaevictis embeddings from .csv
# These embeddings were computed in R after having preprocessed the data using Seurat and denoised it using Rmagic
# To have an uniform map, the embeddings correspond to the merged dataset
full_vae = pd.read_csv('seurat_mice_out/mice_all_red2.csv', header=None)
full_vae = full_vae.set_index(keys=[0])
names = full_vae.index.values.tolist()
names = [c.split('-')[0] for c in names] # remove the -1 suffixes that were added by seurat to handle duplicates
full_vae.index = names
full_vae

In [ ]:
# read sample 1 (DNWT)
aDNWT = scv.read('scvelo_out/DNWT/sample_alignments_BKC62.loom', cache=True)
# preproc
scv.pp.filter_and_normalize(aDNWT, min_shared_counts=20, n_top_genes=3000)
scv.pp.moments(aDNWT, n_pcs=30, n_neighbors=30)
# add dimred
# warning: the order of cells between the imported embeddings and the loom output of velocyto is not the same
dnwt_brcd = aDNWT.obs_names.values.tolist()
dnwt_brcd = [c.split(':')[-1][:-1] for c in dnwt_brcd] # fix cell name formatting
aDNWT.uns['vae'] = {}
n1 = aDNWT.shape[0]
aDNWT.obsm['X_vae'] = full_vae.iloc[:n1,:].loc[dnwt_brcd].values # filter and reorder embedding
# velocity tools
scv.tl.velocity(aDNWT, mode='stochastic')
scv.tl.velocity_graph(aDNWT, basis = None, approx=True)
# latent time
scv.tl.recover_dynamics(aDNWT)
scv.tl.latent_time(aDNWT)
scv.tl.umap(aDNWT)

In [ ]:
# phase portraits of specific genes
# scv.pl.velocity(aDNWT, var_names=['CD4', 'TOP2A'], basis='vae')
# velocity projection
scv.pl.velocity_embedding_stream(aDNWT, basis='vae', size=15, dpi=360, density=4, alpha=1)
scv.pl.scatter(aDNWT, color='latent_time', color_map='gnuplot', basis='vae', size=15, dpi=360, alpha=1)
scv.pl.scatter(aDNWT, color='velocity_pseudotime', color_map='gnuplot', basis='vae', size=15, dpi=360, alpha=1)

In [ ]:
# read sample 2 (DNMUT)
aDNMUT = scv.read('scvelo_out/DNMUT/sample_alignments_TASXR.loom', cache=True)
# preproc
scv.pp.filter_and_normalize(aDNMUT, min_shared_counts=20, n_top_genes=3000)
scv.pp.moments(aDNMUT, n_pcs=30, n_neighbors=30)
# add dimred
# warning: the order of cells between the imported embeddings and the loom output of velocyto is not the same
dnmut_brcd = aDNMUT.obs_names.values.tolist()
dnmut_brcd = [c.split(':')[-1][:-1] for c in dnmut_brcd]
aDNMUT.uns['vae'] = {}
n2 = aDNMUT.shape[0] + n1
aDNMUT.obsm['X_vae'] = full_vae.iloc[n1:n2,:].loc[dnmut_brcd].values
# velocity tools
scv.tl.velocity(aDNMUT, mode='stochastic')
scv.tl.velocity_graph(aDNMUT, basis = None, approx=True)
# latent time
scv.tl.recover_dynamics(aDNMUT)
scv.tl.latent_time(aDNMUT)
scv.tl.umap(aDNMUT)

In [ ]:
# phase portraits of specific genes
# scv.pl.velocity(aDNMUT, var_names=['CD4', 'TOP2A'], basis='vae')
# velocity projection
scv.pl.velocity_embedding_stream(aDNMUT, basis='vae', size=15, dpi=360, density=4, alpha=1)
scv.pl.scatter(aDNMUT, color='latent_time', color_map='gnuplot', basis='vae', size=15, dpi=360, alpha=1)
scv.pl.scatter(aDNMUT, color='velocity_pseudotime', color_map='gnuplot', basis='vae', size=15, dpi=360, alpha=1)

In [ ]:
# read sample 3 (TOTWT)
aTOTWT = scv.read('scvelo_out/TOTWT/sample_alignments_O3KZL.loom', cache=True)
# preproc
scv.pp.filter_and_normalize(aTOTWT, min_shared_counts=20, n_top_genes=3000)
scv.pp.moments(aTOTWT, n_pcs=30, n_neighbors=30)
# add dimred
# warning: the order of cells between the imported embeddings and the loom output of velocyto is not the same
totwt_brcd = aTOTWT.obs_names.values.tolist()
totwt_brcd = [c.split(':')[-1][:-1] for c in totwt_brcd]
aTOTWT.uns['vae'] = {}
n3 = aTOTWT.shape[0] + n2
aTOTWT.obsm['X_vae'] = full_vae.iloc[n2:n3,:].loc[totwt_brcd].values
# velocity tools
scv.tl.velocity(aTOTWT, mode='stochastic')
scv.tl.velocity_graph(aTOTWT, basis = None, approx=True)
# latent time
scv.tl.recover_dynamics(aTOTWT)
scv.tl.latent_time(aTOTWT)
scv.tl.umap(aTOTWT)

In [ ]:
# phase portraits of specific genes
# scv.pl.velocity(aTOTWT, var_names=['CD4', 'TOP2A'], basis='vae')
# velocity projection
scv.pl.velocity_embedding_stream(aTOTWT, basis='vae', size=15, dpi=360, density=4, alpha=1)
scv.pl.scatter(aTOTWT, color='latent_time', color_map='gnuplot', basis='vae', size=15, dpi=360, alpha=1)
scv.pl.scatter(aTOTWT, color='velocity_pseudotime', color_map='gnuplot', basis='vae', size=15, dpi=360, alpha=1)

In [ ]:
# read sample 4 (TOTMUT)
aTOTMUT = scv.read('scvelo_out/TOTMUT/sample_alignments_EDD3M.loom', cache=True)
# preproc
scv.pp.filter_and_normalize(aTOTMUT, min_shared_counts=20, n_top_genes=3000)
scv.pp.moments(aTOTMUT, n_pcs=30, n_neighbors=30)
# add dimred
# warning: the order of cells between the imported embeddings and the loom output of velocyto is not the same
totmut_brcd = aTOTMUT.obs_names.values.tolist()
totmut_brcd = [c.split(':')[-1][:-1] for c in totmut_brcd]
aTOTMUT.uns['vae'] = {}
n4 = aTOTMUT.shape[0] + n3
aTOTMUT.obsm['X_vae'] = full_vae.iloc[n3:n4,:].loc[totmut_brcd].values
# velocity tools
scv.tl.velocity(aTOTMUT, mode='stochastic')
scv.tl.velocity_graph(aTOTMUT, basis = None, approx=True)
# latent time
scv.tl.recover_dynamics(aTOTMUT)
scv.tl.latent_time(aTOTMUT)
scv.tl.umap(aTOTMUT)

In [ ]:
# phase portraits of specific genes
# scv.pl.velocity(aTOTMUT, var_names=['CD4', 'TOP2A'], basis='vae')
# velocity projection
scv.pl.velocity_embedding_stream(aTOTMUT, basis='vae', size=15, dpi=360, density=4, alpha=1)
scv.pl.scatter(aTOTMUT, color='latent_time', color_map='gnuplot', basis='vae', size=15, dpi=360, alpha=1)
# scv.pl.scatter(aTOTMUT, color='velocity_pseudotime', color_map='gnuplot', basis='vae', size=15, dpi=360, alpha=1)

In [ ]:
# Merging .bam files together to apply velocyto on all samples at once
# Command might not work in the jupyter notebook
# If so, ssh into the server via console and access the container by 'docker exec -it scvelo_ahmad /bin/bash'
# Then paste the command and run it (might take a few hours)
samtools merge -o scvelo_out/sample_alignments_merged.bam cellranger/Mice/08-23/DNWT/outs/per_sample_outs/DNWT/count/sample_alignments.bam cellranger/Mice/08-23/DNMUT/outs/per_sample_outs/DNMUT/count/sample_alignments.bam cellranger/Mice/08-23/TOTWT/outs/per_sample_outs/TOTWT/count/sample_alignments.bam cellranger/Mice/08-23/TOTMUT/outs/per_sample_outs/TOTMUT/count/sample_alignments.bam

In [ ]:
!velocyto run -o scvelo_out/ALL scvelo_out/sample_alignments_merged.bam mice_ref/genes.gtf -b velocyto_barcodes/ALL_filt.tsv.gz

In [ ]:
## stochastic mode
# read
aALL = scv.read('scvelo_out/ALL/sample_alignments_merged_3KNCI.loom', cache=True)
# preproc
scv.pp.filter_and_normalize(aALL, min_shared_counts=20, n_top_genes=3000)
scv.pp.moments(aALL, n_pcs=30, n_neighbors=30)
# add dimred
# warning: the order of cells between the imported embeddings and the loom output of velocyto is not the same
all_brcd = aALL.obs_names.values.tolist()
all_brcd = [c.split(':')[-1][:-1] for c in all_brcd]
aALL.uns['vae'] = {}
full_vae_u = full_vae[~full_vae.index.duplicated(keep='first')] # a few dozen cells are duplicated 
aALL.obsm['X_vae'] = full_vae_u.loc[all_brcd].values
# velocity tools
scv.tl.velocity(aALL, mode='stochastic')
scv.tl.velocity_graph(aALL, basis = None, approx=True)
# latent time
scv.tl.recover_dynamics(aALL)
scv.tl.latent_time(aALL)
scv.tl.umap(aALL)

In [ ]:
# phase portraits of specific genes
# scv.pl.velocity(aALL, var_names=['CD4', 'TOP2A'], basis='vae')
# velocity projection
scv.pl.velocity_embedding_stream(aALL, basis='vae', size=15, dpi=360, density=6, alpha=1)
scv.pl.scatter(aALL, color='latent_time', color_map='gnuplot', basis='vae', size=15, dpi=360, alpha=1)
scv.pl.scatter(aALL, color='velocity_pseudotime', color_map='gnuplot', basis='vae', size=15, dpi=360, alpha=1)

In [ ]:
## dynamical mode
# read
aALL = scv.read('scvelo_out/ALL/sample_alignments_merged_3KNCI.loom', cache=True)
# preproc
scv.pp.filter_and_normalize(aALL, min_shared_counts=20, n_top_genes=3000)
scv.pp.moments(aALL, n_pcs=30, n_neighbors=30)
# add dimred
# warning: the order of cells between the imported embeddings and the loom output of velocyto is not the same
all_brcd = aALL.obs_names.values.tolist()
all_brcd = [c.split(':')[-1][:-1] for c in all_brcd]
aALL.uns['vae'] = {}
full_vae_u = full_vae[~full_vae.index.duplicated(keep='first')] # a few dozen cells are duplicated 
aALL.obsm['X_vae'] = full_vae_u.loc[all_brcd].values
# velocity tools
scv.tl.recover_dynamics(aALL)
scv.tl.velocity(aALL, mode='dynamical')
scv.tl.velocity_graph(aALL, basis = None, approx=True)
# latent time
scv.tl.latent_time(aALL)
scv.tl.umap(aALL)

In [ ]:
# phase portraits of specific genes
# scv.pl.velocity(aALL, var_names=['CD4', 'TOP2A'], basis='vae')
# velocity projection
scv.pl.velocity_embedding_stream(aALL, basis='vae', size=15, dpi=360, density=6, alpha=1)
scv.pl.scatter(aALL, color='latent_time', color_map='gnuplot', basis='vae', size=15, dpi=360, alpha=1)
scv.pl.scatter(aALL, color='velocity_pseudotime', color_map='gnuplot', basis='vae', size=15, dpi=360, alpha=1)

In [ ]:
# Non-T-cells and Tgd cells (approx. 2k cells) have been removed to make this barcode file
!velocyto run -o scvelo_out/ALL_T scvelo_out/sample_alignments_merged.bam mice_ref/genes.gtf -b velocyto_barcodes/ALL_filt_onlyT.tsv.gz

In [ ]:
# read
aALLT = scv.read('scvelo_out/ALL_T/sample_alignments_merged_5KJ8Z.loom', cache=True)
# preproc
scv.pp.filter_and_normalize(aALLT, min_shared_counts=20, n_top_genes=3000)
scv.pp.moments(aALLT, n_pcs=30, n_neighbors=30)
# add dimred
# warning: the order of cells between the imported embeddings and the loom output of velocyto is not the same
allt_brcd = aALLT.obs_names.values.tolist()
allt_brcd = [c.split(':')[-1][:-1] for c in allt_brcd]
aALLT.uns['vae'] = {}
full_vae_u = full_vae[~full_vae.index.duplicated(keep='first')] # a few dozen cells are duplicated 
aALLT.obsm['X_vae'] = full_vae_u.loc[allt_brcd].values
# velocity tools
scv.tl.recover_dynamics(aALLT)
scv.tl.velocity(aALLT, mode='stochastic')
scv.tl.velocity_graph(aALLT, basis = None, approx=True)
# latent time
scv.tl.latent_time(aALLT)
scv.tl.umap(aALLT)

In [ ]:
# phase portraits of specific genes
# scv.pl.velocity(aALLT, var_names=['CD4', 'TOP2A'], basis='vae')
# velocity projection
scv.pl.velocity_embedding_stream(aALLT, basis='vae', size=15, dpi=360, density=6, alpha=1)
scv.pl.scatter(aALLT, color='latent_time', color_map='gnuplot', basis='vae', size=15, dpi=360, alpha=1)
scv.pl.scatter(aALLT, color='velocity_pseudotime', color_map='gnuplot', basis='vae', size=15, dpi=360, alpha=1)